In [32]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry 
from std_msgs.msg import Bool
import ipywidgets as widgets
from IPython.display import display
import assignment_2_2023.msg
from assignment_2_2023.msg import PlanningAction
from assignment_2_2023.msg import Info
import actionlib
import actionlib.msg

In [33]:
def on_feedback(action_feedback):
    global canCancel, send_coordinates
    
    # We can preempt the robot only if the state machine is in the "ACTIVE" status
    canCancel = (action_feedback.status.status == action_feedback.status.ACTIVE)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"

In [34]:
def on_result(action_result):
    global canCancel, send_coordinates
    # We can preempt the robot only if the state machine is not in the "SUCCEEDED" or in the "PREEMPTED" status
    canCancel = not(action_result.status.status == action_result.status.SUCCEEDED or action_result.status.status == action_result.status.PREEMPTED)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"
    

In [35]:
def on_odom(msg):
    global info
    pos = msg.pose.pose.position
	
	# Get the current velocity
    vel_linear_x = msg.twist.twist.linear.x
    vel_angular_z = msg.twist.twist.angular.z
    
    # rospy.loginfo("Velocity: %f, %f", vel_linear_x, vel_angular_z)
    
    # Prepare and publish the info message
    info = Info()
    info.x = pos.x
    info.y = pos.y
    info.vel_linear_x = vel_linear_x
    info.vel_angular_z = vel_angular_z

In [36]:
def show_data(a, b, c):
    print('{} {} {}'.format(a, b, c))

In [47]:

def on_coordinates_sent(b):
    global x_coordinate, y_coordinate, canCancel, pub_coordinates, pub_cancel, client, info, vbox
    if(b.description == "Cancel"):
        b.description = "Send Coordinates"
        client.cancel_goal()
        
        
        hbox = widgets.HBox()

        hbox.children = ([widgets.Label("Deleted"), widgets.Label("{:.1f}".format(info.x)), widgets.Label("{:.1f}".format(info.y))])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
    elif(b.description == "Send Coordinates"):
        goal = assignment_2_2023.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x_coordinate.value
        goal.target_pose.pose.position.y = y_coordinate.value
        
        hbox = widgets.HBox()

        hbox.children = ([widgets.Label("Start"), widgets.Label(str(x_coordinate.value)), widgets.Label(str(y_coordinate.value))])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
        rospy.loginfo("Goal set to x = %f, y = %f", goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        client.send_goal(goal)
        print("Coordinates sent with x = %.1f, y = %.1f" % (x_coordinate.value, y_coordinate.value))
        b.description = "Cancel"

In [44]:
global pub_coordinates, pub_cancel, client

rospy.init_node('client_node')

# create a publisher for the coordinates goal topic
pub_coordinates = rospy.Publisher('/goal/coordinates', Info, queue_size=1)

# create a publisher for the cancel goal topic
pub_cancel = rospy.Publisher('/goal/cancel', Bool, queue_size=1)

# Subscriber for feedback/status
sub_feedback = rospy.Subscriber("/reaching_goal/feedback", assignment_2_2023.msg.PlanningActionFeedback, on_feedback)
    
# Subscriber for result
sub_result = rospy.Subscriber("/reaching_goal/result", assignment_2_2023.msg.PlanningActionResult, on_result)

# sub_odom is a subscriber that receives Odometry data
sub_odom = rospy.Subscriber("/odom", Odometry, on_odom)

client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()



ciao


True

In [48]:
global x_coordinate, y_coordinate, send_coordinates, vbox
x_coordinate = widgets.FloatSlider( 
value=7.0,
min=0.0, 
max=10.0, 
step=0.1,
description='X:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
y_coordinate = widgets.FloatSlider( 
value=7.0,
min=0.0, 
max=10.0, 
step=0.1,
description='Y:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
display(x_coordinate, y_coordinate)

send_coordinates = widgets.Button(description="Send Coordinates")
output = widgets.Output()
display(send_coordinates, output)
send_coordinates.on_click(on_coordinates_sent)
# Display the tables
vbox = widgets.VBox()
hbox = widgets.HBox()

hbox.children = ([widgets.Label("Operation"), widgets.Label("X"), widgets.Label("Y")])
vbox.children = tuple([hbox,])
display(vbox)


FloatSlider(value=7.0, continuous_update=False, description='X:', max=10.0)

FloatSlider(value=7.0, continuous_update=False, description='Y:', max=10.0)

Button(description='Send Coordinates', style=ButtonStyle())

Output()

[INFO] [1711998273.553755, 1476.966000]: Goal set to x = 7.000000, y = 7.000000
Coordinates sent with x = 7.0, y = 7.0


In [40]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))
out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
widgets.HBox([widgets.VBox([a, b, c]), out])


In [41]:
vbox = widgets.VBox()
hbox = widgets.HBox()

hbox.children = ([widgets.Label("Operation"), widgets.Label("X"), widgets.Label("Y")])
vbox.children = tuple([hbox,])



In [42]:
hbox = widgets.HBox()

hbox.children = ([widgets.Label("Start"), widgets.Label("9.0"), widgets.Label("1.0")])
vbox.children = vbox.children + tuple([hbox,])
vbox

